## Importar librerias

In [18]:
# Importar librerias necesarias
import janitor
import matplotlib.pyplot as plt
import missingno
import numpy as np
import pandas as pd
import pyreadr
import seaborn as sns
import session_info
import upsetplot


## Configurar el aspecto general de las graficas del proyecto

In [19]:
%matplotlib inline

sns.set(rc={'figure.figsize':(10,10)})

sns.set_style("whitegrid")

### Operar con valores faltantes

#### Python

In [20]:
print(None or True,
      None or True,
      None == None,
      None is None,
      type(None)
      )

True True True True <class 'NoneType'>


### Nympy

In [21]:
print(np.nan or True,
      np.nan == np.nan,
      np.nan is np.nan,
      np.nan /  2,
      type(np.nan),
      type(np.nan) is float,
      sep='\n'
      )

nan
False
True
nan
<class 'float'>
True


### Pandas


In [22]:
test_missing_df = pd.DataFrame.from_dict(
    data = dict(
        x= [0,1, np.nan, np.nan, None],
        y= [0,1, pd.NA, np.nan, None]
        )
    )

test_missing_df

,x,y
0,0.0,0
1,1.0,1
2,NaN,<NA>
3,NaN,NaN
4,NaN,None


## Cargar los conjuntos de datos

#### Pima Indians Diabetes

In [31]:
diabetes_df = pd.read_csv("diabetes.csv")
diabetes_df.head(10)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
5,5,116,74,0,0,25.6,0.201,30,0
6,3,78,50,32,88,31.0,0.248,26,1
7,10,115,0,0,0,35.3,0.134,29,0
8,2,197,70,45,543,30.5,0.158,53,1
9,8,125,96,0,0,0.0,0.232,54,1


## Usando mi propia libreria

In [32]:
from libreria_eda import analyze_data

In [33]:
df = pd.read_csv("diabetes.csv")

resultados = analyze_data(df)

resultados

{'Dataset Information': {'Number of rows': 768, 'Number of columns': 9},
 'Missing Values Summary': {'Total missing values': np.int64(0),
  'Percentage of missing values': '0.00%'},
 'Column-wise Missing Values':                           Missing Values  Percentage Missing
 Pregnancies                            0                 0.0
 Glucose                                0                 0.0
 BloodPressure                          0                 0.0
 SkinThickness                          0                 0.0
 Insulin                                0                 0.0
 BMI                                    0                 0.0
 DiabetesPedigreeFunction               0                 0.0
 Age                                    0                 0.0
 Outcome                                0                 0.0,
 'Rows with Missing Values': {'Number of rows with missing values': 0,
  'Percentage of rows with missing values': '0.00%',
  'Rows with missing values': []},
 'Genera